#Minimum Code Requirement

In [0]:
!pip install Pillow==4.1.1
!pip install "fastai==0.7.0"
!pip install torchtext==0.2.3
!apt-get -qq install -y libsm6 libxext6 && pip install -q -U opencv-python
import cv2
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
!apt update && apt install -y libsm6 libxext6

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'
!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
import torch
!pip install image

%matplotlib inline

In [0]:
!wget http://files.fast.ai/data/dogscats.zip
!unzip dogscats.zip -d data

In [0]:
from fastai.conv_learner import *
PATH = 'data/dogscats/'
size = 224                         # size of image
batch_size = 64
arch = resnet50

In [10]:
'''
# Set up our transforms, data, and learner
# Do our initial learning for our final layer
# Note we do not call precompute = true. This is just an optimization that lets us some weights that have been computed before. 
# We can always leave it out though.
'''


tfms = tfms_from_model(arch,size,aug_tfms = transforms_side_on,max_zoom =1.1)
data = ImageClassifierData.from_paths(PATH,tfms=tfms,bs=batch_size) # if not in format of cats dogs train test 
                                                                     # give train and test locn e.g 'train',test_val ='test'
learn = ConvLearner.pretrained(resnet50,data)
%time learn.fit(0.01,3,cycle_len=1)


Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.torch/models/resnet50-19c8e357.pth
100%|██████████| 102502400/102502400 [00:03<00:00, 26527591.32it/s]


epoch      trn_loss   val_loss   accuracy   
    0      0.053495   0.029474   0.9875    
 95%|█████████▍| 341/360 [1:44:48<06:24, 20.21s/it, loss=0.0359]


KeyboardInterrupt



In [8]:
# now training other layers also
# Unfreeze the other layers and learn again using differential learning rates
# (Lower rates for more general layers)
learn.unfreeze()

# this will cause batch_normalization moving averages to not be updated
learn.bn_freeze(True)

diff_rate = np.array([1e-5,1e-4,1e-2])

%time learn.fit(diff_rate,1,cycle_len =1)



NameError: ignored

In [0]:
# Use test time augmentation to get our predictions
# Recall test time augmentation means transforming our test images to increase
# the chance that they are identified correctly

%time log_predictions,y = learn.TTA()   # as our predictions come out in log

In [0]:
y

In [0]:
# 2000 images, can be in one of two classes, and we take a prediction for 1 image + 4 augmentations
log_predictions.shape

In [0]:
predictions = np.exp(log_predictions)

In [0]:
prob = np.mean(predictions,0)
metrics.log_loss(y,probabilities)


In [0]:
accuracy(np.mean((log_predictions),0), y)